In [ ]:
from pathlib import Path

import pandas as pd
from util_func import *
from xlsxwriter.utility import xl_rowcol_to_cell

In [ ]:
survey_processed_dir = Path(
    "Q:/Data/Surveys/HouseholdSurveys/MTC-SFCTA2022/Processed_20240329"
)
reformat_dir = survey_processed_dir / "02-reformat"
tour_extract_wkday_dir = survey_processed_dir / "03-tour_extract" / "wt_wkday"
tour_extract_allwk_dir = survey_processed_dir / "03-tour_extract" / "wt_7day"
out_dir = survey_processed_dir / "04b-summary_notebooks"
out_dir.mkdir(exist_ok=True)
out_filepath = out_dir / "01_AllTrips_Mode.xlsx"

In [ ]:
## Process trip records
trip_df = pd.read_csv(tour_extract_wkday_dir / "trip-assign_day.csv")
trip_df = link_dt(trip_df)
trip_df["count"] = 1
trip_df = trip_df[(trip_df["trexpfac"] > 0) & (trip_df["mode"] > 0)]
trip_df = trip_df[(trip_df["otaz"] > 0) & (trip_df["dtaz"] > 0)]

In [ ]:
col_dict = {
    "mode": {
        "desc": "Mode",
        "col": "mode",
        "vals": range(1, 10),
        "labels": [
            "7_Wk",
            "8_Bk",
            "1_DA",
            "2_SOV2",
            "3_SOV3",
            "5_WTrn",
            "6_DTrn",
            "9_SBus",
            "4_TNC",
        ],
    },
    "day": {
        "desc": "DOW",
        "col": "day",
        "vals": range(6, 8),
        "labels": ["6_Sat", "7_Sun"],
    },
}

In [ ]:
writer = pd.ExcelWriter(out_filepath, engine="xlsxwriter")
workbook = writer.book
# format1 = workbook.add_format({"num_format": "#,##0.0"})

In [ ]:
row = 0
sname = "Weekday"
d1_dict = col_dict["mode"]

title = "All Trips by " + d1_dict["desc"]
tab = prep_data_1d(
    trip_df,
    d1_dict["desc"],
    d1_dict["col"],
    "trexpfac",
    d1_dict["vals"],
    d1_dict["labels"],
)
row = write_to_excel(writer, tab, sname, title, row)

title = "Column Shares by " + d1_dict["desc"]
row = write_to_excel(writer, getSharesIdx(tab.copy()), sname, title, row)

tab2 = tab.copy()
tab2.iloc[-1, -1] = tab2.iloc[-1, 0]

title = "Column Shares 95% CI by " + d1_dict["desc"]
row = write_to_excel(writer, getSharesIdxCI95(tab.copy()), sname, title, row)

tab_range = xl_rowcol_to_cell(row, 1) + ":" + xl_rowcol_to_cell(row, tab.shape[1])
_ = writer.sheets[sname].set_column(tab_range, 11)  # , format1)

In [ ]:
# Process all week trip records
trip_df = pd.read_csv(tour_extract_allwk_dir / "trip-assign_day.csv")
trip_df = link_dt(trip_df)
trip_df["count"] = 1
trip_df = trip_df[(trip_df["trexpfac"] > 0) & (trip_df["mode"] > 0)]
trip_df = trip_df[(trip_df["otaz"] > 0) & (trip_df["dtaz"] > 0)]

In [ ]:
row = 0
sname = "Weekend"
d1_dict = col_dict["mode"]
d2_dict = col_dict["day"]

wt_cols = ["count", "trexpfac"]
wt_desc = ["(Unweighted)", "(Weighted)"]

for wc, wd in zip(wt_cols, wt_desc):
    title = "All Trips by " + d2_dict["desc"] + " and " + d1_dict["desc"] + " " + wd
    tab = prep_data_2d(
        trip_df,
        d1_dict["col"],
        d1_dict["vals"],
        d1_dict["labels"],
        d2_dict["col"],
        d2_dict["vals"],
        d2_dict["labels"],
        wc,
    )
    row = write_to_excel(writer, tab.astype("float64"), sname, title, row)

    if wc == "count":
        tab2 = tab.copy()
    else:
        tab2.iloc[:-1, :-1] = tab.iloc[:-1, :-1]

    title = "Column Shares by " + d2_dict["desc"] + " and " + d1_dict["desc"] + " " + wd
    row = write_to_excel(writer, getSharesIdx(tab.copy()), sname, title, row)

    title = (
        "Column Shares 95% CI by "
        + d2_dict["desc"]
        + " and "
        + d1_dict["desc"]
        + " "
        + wd
    )
    row = write_to_excel(writer, getSharesIdxCI95(tab2.copy()), sname, title, row)
tab_range = xl_rowcol_to_cell(row, 1) + ":" + xl_rowcol_to_cell(row, tab.shape[1])
_ = writer.sheets[sname].set_column(tab_range, 11)  # , format1)

In [ ]:
writer.close()